In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

def waterfall_plot(ez_plots, epsR, res, time_points, rel_source_pos, rel_mat_beg, rel_mat_end, ref_point, trans_point):
    '''Waterfall ploting for the simulations. Can plot multiple variations.'''
    # Create waterfall plot axis
    xsize = len(ez_plots[0][0])
    source_pos = math.floor(rel_source_pos*xsize)
    mat_begin = math.floor(rel_mat_beg * xsize)
    mat_end = math.floor(rel_mat_end * xsize)
    fig, ax = plt.subplots(figsize=(12, 10))
    #Downsampling to speed up if grid is too fine
    if res >= 10:
        x_downsample = math.floor(res / 10)
    else:
        x_downsample = 1

    x = np.arange(xsize) 
    vertical_offset = 2.0  # Vertical spacing between traces
    total_height = len(ez_plots[0]) * vertical_offset
    # epsR background with alpha scaled by relative permittivity 

    epsR_norm = (epsR - 1.0) / (epsR.max() - 1.0)   # epsR=1 → 0, max → 1
    total_height = len(ez_plots[0]) * vertical_offset
    epsR_plot = epsR_norm[::x_downsample]
    x_plot = x[::x_downsample]

    for i in range(len(x_plot) - 1): 
        if epsR_plot[i] > 0:
            ax.fill_between([x_plot[i], x_plot[i+1]], 0, total_height, color='orange', alpha=0.25 * epsR_plot[i], linewidth=0)

    color_ind = 0
    cmap = plt.get_cmap("viridis")
    for ez_history in ez_plots:
        color = cmap(color_ind / (len(ez_plots)))
        # Plot each time step offset vertically
        for i, (t, ez_snapshot) in enumerate(zip(time_points, ez_history)):
            # Offset each trace vertically
            y_offset = i * vertical_offset
            ax.plot(x[::x_downsample], ez_snapshot[::x_downsample] + y_offset, color = color, linewidth=0.8, alpha=0.7)
            
            # Optional: add baseline for each trace
            #ax.axhline(y=y_offset, color='gray', linewidth=0.3, alpha=0.3, linestyle='--')
            
            # Optional: label some time steps on the right
            if i % 5 == 0:  # Label every 5th trace
                ax.text(xsize + 2, y_offset, f't={t}', fontsize=8, va='center')
        color_ind += 1
        
    # Mark source location
    ax.axvline(x=source_pos, color='green', linestyle='--', linewidth=1.5, alpha=0.5, label='Source')
    ax.axvline(x=mat_begin, color='purple', linestyle='--', linewidth=1.5, alpha=0.5, label='Material Begin')
    ax.axvline(x=mat_end, color='purple', linestyle='--', linewidth=1.5, alpha=0.5, label='Material End')
    ax.axvline(x=math.floor(ref_point*xsize), color='yellow', linestyle='--', linewidth=1.5, alpha=0.5, label='Reflection Measurement Point')
    ax.axvline(x=math.floor(trans_point*xsize), color='orange', linestyle='--', linewidth=1.5, alpha=0.5, label='Transmission Measurement Point')

    
    ax.set_xlabel('Position', fontsize=12)
    ax.set_ylabel('Time →', fontsize=12)
    ax.set_title('Stacked Waterfall Plot - FDTD Ez Field', fontsize=14)
    ax.set_xlim(0, xsize)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.2)

    plt.tight_layout()
    #plt.savefig('fdtd_stacked_waterfall.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("Waterfall Plot Generated")


This is a bit messy. I defined a outer function, so I could run it many times. There's also a plot_spectra_from_series() function that I was testing out to find the reflection and absorption spectra, but I was struggling with some of the signal processing needed, so that doesn't work well currently.

In [ ]:



import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.ndimage import gaussian_filter1d

def fdtd_super(freq_c = 0.05, amp = 2.0, interface = "random_smooth", res = 1.0, plot= True, nonlinear = False ):
    # All of the physical parameters. Changing these changes the situation being simulated
    # Physical simulation parameters
    SIZE = 250
    IMP0 = 377.0


    MAX_TIME = 1000

    # Ricker wavelet parameters
    FREQ_C = freq_c  # Central frequency (cycles per grid point/time step)
    T0 = 1.0 / (FREQ_C)  # Pulse delay
    AMPLITUDE = amp
    RELATIVE_SOURCE_POS = 0.20
    RELATIVE_MATERIAL_BEG = 0.40
    RELATIVE_MATERIAL_END = 0.80
    REF_POINT = 0.10
    TRANS_POINT = 0.90


    #Ricker wavelet source (resolution invariant)
    def ricker_wavelet(t, f_c=FREQ_C, amp = AMPLITUDE):
        pi2 = (np.pi * f_c * t)**2
        return amp * (1 - 2 * pi2) * np.exp(-pi2)

    def generate_grid(struct = "simple_barrier", res = 1, nonlin = False):
        '''Generates epsilon distribution'''
        xsize = math.floor(SIZE*res)
        sep1 = math.floor(xsize * RELATIVE_MATERIAL_BEG)
        sep2 = math.floor(xsize * RELATIVE_MATERIAL_END)
        ez = np.zeros(xsize)
        hy = np.zeros(xsize)
        alpha = np.zeros(xsize)
        epsR = np.ones(xsize)
        if struct == "empty":
            print("Generating empty grid...")
        elif struct == "simple_barrier":
            sep1 = math.floor(xsize * .4)
            epsR[sep1:sep2] = 9
            if nonlin == True:
                alpha[sep1:sep2] = 0.001 # Change this to ramp up nonlinearities
        elif struct == "random_smooth":
            rand_vals = np.random.rand(xsize)
            smooth_rand = gaussian_filter1d(rand_vals, sigma=10)

            eps_min = 1.0
            eps_max = 12.0
            epsR_smooth = eps_min + (eps_max - eps_min) * (smooth_rand - smooth_rand.min()) / (smooth_rand.max() - smooth_rand.min())

            # Assign only after sep
            epsR[sep1:sep2] = epsR_smooth[sep1:sep2]
            if nonlin == True:
                alpha[sep1:sep2] = epsR[sep1:sep2] * 1e-3
        
        elif struct == "random_spiked":
            rand_vals = np.random.rand(xsize)
            smooth_rand = gaussian_filter1d(rand_vals, sigma=10)

            eps_min = 1.0
            eps_max = 5.0
            epsR_smooth = eps_min + (eps_max - eps_min) * (smooth_rand - smooth_rand.min()) / (smooth_rand.max() - smooth_rand.min())

            # Assign only after sep
            epsR[sep1:sep2] = epsR_smooth[sep1:sep2]

            #random spikes
            num_spikes = 5
            spike_width = len(epsR) // 40
            spike_min = 8
            spike_max = 25

            for _ in range(num_spikes):
                # choose random center after sep
                center = np.random.randint(sep1, sep2 - spike_width)

                spike_value = np.random.uniform(spike_min, spike_max)

                epsR[center:center+spike_width] = spike_value
            if nonlin == True:
                alpha[sep1:sep2] = epsR[sep1:sep2] * 1e-3
                    
                
        elif nonlin == False:
            alpha = np.zeros(xsize)

        else:
            print("Structure not found.")
        return ez, hy, epsR, alpha, res

    def fdtd_tfsf(ez, hy, epsR, alpha, res = 1, rel_source_pos = RELATIVE_SOURCE_POS, bound_cond = "1ABC", plot=False, nonlin = False, small_nonlin = True):
        
        '''Core FDTD Engine. res is a resolution multiplier. nonlin adds Kerr nonlinearity update. When small linear is false the cubic will be solved explicitly (takes much longer)'''
        
        tsize = math.floor(MAX_TIME * res)
        xsize = math.floor(SIZE * res)
        
        if bound_cond == "CPML":
            # Pad all arrays
            npml = math.floor(20*res)
            xsize_inner = xsize
            xsize = xsize + 2 * npml
            ez = np.zeros(xsize)
            hy = np.zeros(xsize)
            epsR = np.pad(epsR, npml, mode='edge')
            alpha = np.pad(alpha, npml, mode='edge')

            # Probe positions shifted into padded grid
            refl_probe_cell = math.floor(xsize_inner * REF_POINT) + npml
            trans_probe_cell = math.floor(xsize_inner * TRANS_POINT) + npml
            source_pos = math.floor(rel_source_pos * xsize_inner) + npml
        else:
            source_pos = math.floor(rel_source_pos*xsize)
            refl_probe_cell = math.floor(xsize * REF_POINT)
            trans_probe_cell = math.floor(xsize * TRANS_POINT)


        if bound_cond == "CPML":
            sigma = np.zeros(xsize)
            kappa = np.ones(xsize)
            alpha_pml = np.zeros(xsize)

            m = 3
            kappa_max = 5.0
            alpha_max = 0.1
            R = 1e-6
            sigma_max = -(m + 1) * np.log(R) / (2 * npml)

            # Left PML (cells 0..npml-1)
            for i in range(npml):
                x = (npml - i) / npml
                sigma[i] = sigma_max * x**m
                kappa[i] = 1 + (kappa_max - 1) * x**m
                alpha_pml[i] = alpha_max * (1 - x)

            # Right PML (cells xsize-npml..xsize-1)
            for i in range(xsize - npml, xsize):
                x = (i - (xsize - npml)) / npml
                sigma[i] = sigma_max * x**m
                kappa[i] = 1 + (kappa_max - 1) * x**m
                alpha_pml[i] = alpha_max * (1 - x)

            b = np.exp(-(sigma / kappa + alpha_pml))
            c = sigma * (b - 1) / (kappa * (sigma + kappa * alpha_pml) + 1e-16)

            psi_E = np.zeros(xsize)
            psi_H = np.zeros(xsize)
        
        # Store field history for plotting
        downsample = tsize // 25 # Store every 10th time step to reduce clutter
        ez_history = []
        hy_history = []
        time_points = []
        source_data =[]
        epsRe = epsR.copy() #Effective epsilon
        refl_series = np.zeros(tsize)
        trans_series = np.zeros(tsize)
        
        for qTime in range(tsize):
            t = qTime / res #Adjusts source based on resolution for invariance
            if bound_cond == "1ABC":
                #ABC for hy
                hy[-1] = hy[-2]
            
                # Update magnetic field (vectorized)
                hy[:-1] += (ez[1:] - ez[:-1]) / IMP0

            if bound_cond == "CPML":
                dEz = ez[1:] - ez[:-1]

                psi_H[:-1] = b[1:]*psi_H[:-1] + c[1:]*dEz
                hy[:-1] += ((1/kappa[1:])*dEz + psi_H[:-1]) / IMP0

            # TFSF Correction
            hy[source_pos-1] -= ricker_wavelet(t - T0) / IMP0
            if bound_cond != "CPML":
                # ABC for ez
                ez[0] = ez[1]
            
            if nonlin == True:
                if small_nonlin == True: #Approximation for small linearities
                    epsRe[1:] = epsR[1:] + alpha[1:] * ez[1:]**2
                    # Update electric field (vectorized)
                    if bound_cond == "CPML":
                        dHy = hy[1:] - hy[:-1]

                        psi_E[1:] = b[1:]*psi_E[1:] + c[1:]*(dHy)

                        curl_term = (1/kappa[1:])*(dHy) + psi_E[1:]

                        ez[1:] += curl_term * IMP0 / epsRe[1:]
                    else:
                        ez[1:] += (hy[1:] - hy[:-1]) * IMP0 / epsRe[1:]
                else:
                    dHy = hy[1:] - hy[:-1]
                    # True Kerr: solve cubic eps_r(E) * E = D for all points (complicated slow thing to do)
                    if bound_cond == "CPML":
                        # Update CPML memory variable
                        psi_E[1:] = b[1:] * psi_E[1:] + c[1:] * (dHy )

                        # Modified curl with PML stretching
                        curl_term = (1 / kappa[1:]) * (dHy) + psi_E[1:]

                    else:  # 1ABC (original)
                        curl_term = dHy
                    # Compute D (Ez at next step if explicit, or you can include curl(H) term)
                    D = epsR[1:] * ez[1:] + curl_term * IMP0

                    # Initial guess for E: previous Ez
                    E = ez[1:].copy()

                    # Newton-Raphson iteration
                    max_iter = 20
                    tol = 1e-12

                    for _ in range(max_iter):
                        f = alpha[1:] * E**3 + epsR[1:] * E - D        # cubic: alpha E^3 + epsR0 E - D = 0
                        df = 3 * alpha[1:] * E**2 + epsR[1:]          # derivative
                        delta = f / df
                        E -= delta
                        if np.all(np.abs(delta) < tol):
                            break

                    ez[1:] = E
            else:
                if bound_cond == "CPML":
                    dHy = hy[1:] - hy[:-1]

                    psi_E[1:] = b[1:]*psi_E[1:] + c[1:]*(dHy)

                    ez[1:] += ( (1/kappa[1:])*(dHy) + psi_E[1:] ) * IMP0 / epsR[1:]
                else:
                    ez[1:] += (hy[1:] - hy[:-1]) * IMP0 / epsR[1:]
            
            # TFSF correction
            ez[source_pos] += ricker_wavelet(t + 0.5 - T0)
            
            #Store Reflection Probe Point
            refl_series[qTime] = ez[refl_probe_cell]
            #Store Transmission Probe Point
            trans_series[qTime] = ez[trans_probe_cell]
            # Store snapshots
            if qTime % downsample == 0:
                if bound_cond == "CPML":
                    ez_history.append(ez[npml:npml+xsize_inner].copy())
                    hy_history.append(hy[npml:npml+xsize_inner].copy())
                else:
                    ez_history.append(ez.copy())
                    hy_history.append(hy.copy())
                time_points.append(qTime)

            if qTime == 161:
                source_data.append(ez[npml:npml+xsize_inner].copy())
                source_data.append(hy[npml:npml+xsize_inner].copy())
        if plot == True:
            ezlist = []
            ezlist.append(ez_history)
            waterfall_plot(ezlist, epsR[npml:npml+xsize_inner], res, time_points, rel_source_pos, RELATIVE_MATERIAL_BEG, RELATIVE_MATERIAL_END, REF_POINT, TRANS_POINT)
        
        return ez, hy, ez_history, hy_history, time_points, refl_series, trans_series, source_data



    def plot_spectra_from_series(refl_series, trans_series, inc_series):
        N = len(refl_series)
        
        window = np.hanning(N)
        pad_factor = 4
        refl_freq  = np.fft.fft(refl_series  * window, n=N*pad_factor)
        trans_freq = np.fft.fft(trans_series * window, n=N*pad_factor)
        inc_freq   = np.fft.fft(inc_series   * window, n=N*pad_factor)
        freq_bins  = np.fft.fftfreq(N * pad_factor, 1.0)
        pos_idx = freq_bins > 0
        freq_bins = freq_bins[pos_idx]

        inc_mag = np.abs(inc_freq[pos_idx])
        threshold = 1e-3 * inc_mag.max()
        valid = (inc_mag > threshold) & (freq_bins > 0.01) & (freq_bins < 0.35) #Nyquist frequency is 0.5
        #valid =  (freq_bins > 0.01) & (freq_bins < 0.2)

        R = np.abs(refl_freq[pos_idx] / inc_freq[pos_idx])**2
        T = np.abs(trans_freq[pos_idx] / inc_freq[pos_idx])**2
        total = T+R

        fig, ax = plt.subplots()
        ax.plot(freq_bins[valid], R[valid], label='Reflection', color='blue')
        ax.plot(freq_bins[valid], T[valid], label='Transmission', color='red')
        ax.plot(freq_bins[valid], total[valid], label='T+R', color='green')
        
        ax.set_xlabel('Normalized frequency')
        ax.set_ylabel('Magnitude')
        ax.legend()
        plt.show()

    ez, hy, epsR, alpha, res = generate_grid(res=res, nonlin=nonlinear, struct=interface)
    output = fdtd_tfsf(ez = ez, hy=hy, epsR=epsR, alpha=alpha, res=res, plot=plot, nonlin= nonlinear, small_nonlin=True, bound_cond="CPML")
    return output[2], output[3], output[4], output[7], epsR, alpha
    
# if __name__ == "__main__":
#     ez, hy, epsR, alpha, res = generate_grid(res=8, nonlin=True, struct="simple_barrier")
#     output = fdtd_tfsf(ez = ez, hy=hy, epsR=epsR, alpha=alpha, res=res, plot=True, nonlin= False, small_nonlin=True, bound_cond="CPML")
    # ez, hy, epsR, alpha, res = generate_grid(res=8, nonlin=False, struct="empty")
    # inc_series = fdtd_tfsf(ez = ez, hy = hy, epsR = epsR, alpha = alpha, res = res, plot = True, nonlin = False, small_nonlin = True, bound_cond="CPML")[5]
    # plot_spectra_from_series(refl_series=output[4], trans_series=output[5], inc_series=inc_series)

    

Also a bit messy currently. A Larger function for generating many samples.

In [ ]:
def fdtd_data_gen(n=1000):
    samples = []
    freq_c_min = 0.05
    freq_c_max = 0.07
    amp_min = 1
    amp_max = 10
    interface = "random_spiked"
    nonlinear = True
    for i in range(n):
        freq_c = np.random.uniform(freq_c_min,freq_c_max)
        amp = np.random.uniform(amp_min,amp_max)
        ez_history, hy_history, time_points, source_prop, epsR, alpha = fdtd_super(res=4, freq_c=freq_c, amp = amp, interface=interface, plot=False, nonlinear=nonlinear)
        sample = {
            "ez_history": ez_history,
            "hy_history": hy_history,
            "time_points": time_points,
            "source_prop": source_prop,
            "epsR": epsR,
            "alpha": alpha
        }

        samples.append(sample)
        if i % 5 == 0:
            print(f"{i}/{n} Samples Generated")
    nonlineartag = ""
    if nonlinear == True:
        nonlineartag = "_nonlinear"
    file_name = "fdtdData_" + interface + nonlineartag + "_SET1"
    np.savez_compressed(file_name, samples=samples)
    print("Data saved to working directory as " + file_name + ".npz!")


fdtd_data_gen()